In [1]:
import os
import re
import shutil
import string

from collections import Counter

import pandas as pd
import numpy as np

import sklearn

from sklearn.model_selection import train_test_split

In [2]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_url(text): 
    url_pattern  = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    return url_pattern.sub(r'', text)
 # converting return value from list to string



def clean_text(text ): 
    delete_dict = {sp_character: '' for sp_character in string.punctuation} 
    delete_dict[' '] = ' ' 
    table = str.maketrans(delete_dict)
    text1 = text.translate(table)
    #print('cleaned:'+text1)
    textArr= text1.split()
    text2 = ' '.join([w for w in textArr if ( not w.isdigit() and  ( not w.isdigit() and len(w)>2))]) 
    
    return text2.lower()

In [3]:
def get_sentiment(sentiment):
    if sentiment == 'positive':
        return 2
    elif sentiment == 'negative':
        return 1
    else:
        return 0

In [4]:
train_data= pd.read_csv("train.csv")
train_data.dropna(axis = 0, how ='any',inplace=True) 
train_data['Num_words_text'] = train_data['text'].apply(lambda x:len(str(x).split())) 
mask = train_data['Num_words_text'] >2
train_data = train_data[mask]
print('-------Train data--------')
print(train_data['sentiment'].value_counts())
print(len(train_data))
print('-------------------------')
max_train_sentence_length  = train_data['Num_words_text'].max()


train_data['text'] = train_data['text'].apply(remove_emoji)
train_data['text'] = train_data['text'].apply(remove_url)
train_data['text'] = train_data['text'].apply(clean_text)

train_data['label'] = train_data['sentiment'].apply(get_sentiment)

test_data= pd.read_csv("test.csv")
test_data.dropna(axis = 0, how ='any',inplace=True) 
test_data['Num_words_text'] = test_data['text'].apply(lambda x:len(str(x).split())) 

max_test_sentence_length  = test_data['Num_words_text'].max()

mask = test_data['Num_words_text'] >2
test_data = test_data[mask]

print('-------Test data--------')
print(test_data['sentiment'].value_counts())
print(len(test_data))
print('-------------------------')

test_data['text'] = test_data['text'].apply(remove_emoji)
test_data['text'] = test_data['text'].apply(remove_url)
test_data['text'] = test_data['text'].apply(clean_text)

test_data['label'] = test_data['sentiment'].apply(get_sentiment)

print('Train Max Sentence Length :'+str(max_train_sentence_length))
print('Test Max Sentence Length :'+str(max_test_sentence_length))

-------Train data--------
neutral     10704
positive     8375
negative     7673
Name: sentiment, dtype: int64
26752
-------------------------
-------Test data--------
neutral     1376
positive    1075
negative     983
Name: sentiment, dtype: int64
3434
-------------------------
Train Max Sentence Length :33
Test Max Sentence Length :32


In [5]:
train_data.head(10)

,textID,text,selected_text,sentiment,Num_words_text,label
0,cb774db0d1,have responded were going,"I`d have responded, if I were going",neutral,7,0
1,549e992a42,sooo sad will miss you here san diego,Sooo SAD,negative,10,1
2,088c60f138,boss bullying,bullying me,negative,5,1
3,9642c003ef,what interview leave alone,leave me alone,negative,5,1
4,358bd9e861,sons why couldnt they put them the releases al...,"Sons of ****,",negative,14,1
5,28b57f3990,some shameless plugging for the best rangers f...,http://www.dothebouncy.com/smf - some shameles...,neutral,12,0
6,6e0c6d75b1,2am feedings for the baby are fun when all smi...,fun,positive,14,2
8,e050245fbd,both you,Both of you,neutral,3,0
9,fc2cbefa9d,journey wow just became cooler hehe that possible,Wow... u just became cooler.,positive,10,2
10,2339a9b08b,much love hopeful reckon the chances are minim...,"as much as i love to be hopeful, i reckon the ...",neutral,23,0


In [6]:
test_data.head(10)

,textID,text,sentiment,Num_words_text,label
0,f87dea47db,last session the day,neutral,6,0
1,96d74cb729,shanghai also really exciting precisely skyscr...,positive,15,2
2,eee518ae67,recession hit veronique branquinho she has qui...,negative,13,1
4,33987a8ee5,like,positive,5,2
5,726e501993,thats great weee visitors,positive,4,2
6,261932614e,think everyone hates here lol,negative,8,1
7,afa11da83f,soooooo wish could but school and myspace comp...,negative,13,1
8,e64208b4ef,and within short time the last clue all them,neutral,12,0
9,37bcad24ca,what did you get day alright havent done anyth...,neutral,18,0
10,24c92644a4,bike was put holdshould have known that argh t...,negative,12,1


In [7]:
X_train, X_valid, Y_train, Y_valid= train_test_split(train_data['text'].tolist(),\
                                                      train_data['label'].tolist(),\
                                                      test_size=0.2,\
                                                      stratify = train_data['label'].tolist(),\
                                                      random_state=0)


print('Train data len:'+str(len(X_train)))
print('Class distribution'+str(Counter(Y_train)))


print('Valid data len:'+str(len(X_valid)))
print('Class distribution'+ str(Counter(Y_valid)))

print('Test data len:'+str(len(test_data['text'].tolist())))
print('Class distribution'+ str(Counter(test_data['label'].tolist())))


train_dat =list(zip(Y_train,X_train))
valid_dat =list(zip(Y_valid,X_valid))
test_dat=list(zip(test_data['label'].tolist(),test_data['text'].tolist()))

Train data len:21401
Class distributionCounter({0: 8563, 2: 6700, 1: 6138})
Valid data len:5351
Class distributionCounter({0: 2141, 2: 1675, 1: 1535})
Test data len:3434
Class distributionCounter({0: 1376, 2: 1075, 1: 983})


In [8]:
import torch
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

C:\Users\Ciner\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')
train_iter = train_dat
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [10]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) 

In [11]:
text_pipeline('here is the an example')

[62, 0, 1, 0, 12881]

In [12]:
label_pipeline('1')

1

In [11]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

#train_iter =train_dat
#dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

In [12]:
from torch import nn
import torch.nn.functional as F

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc1 = nn.Linear(embed_dim,64)
        self.fc2 = nn.Linear(64,16)
        self.fc3 = nn.Linear(16, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc1.weight.data.uniform_(-initrange, initrange)
        self.fc1.bias.data.zero_()
        self.fc2.weight.data.uniform_(-initrange, initrange)
        self.fc2.bias.data.zero_()
        self.fc3.weight.data.uniform_(-initrange, initrange)
        self.fc3.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        x = F.relu(self.fc1(embedded))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [15]:
train_iter1 = train_dat
num_class = len(set([label for (label, text) in train_iter1]))
print(num_class)
vocab_size = len(vocab)
emsize = 128
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

3


In [16]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text, offsets)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [17]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 10 # epoch
LR =10  # learning rate
BATCH_SIZE = 16 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None

train_iter2 = train_dat
test_iter2 =test_dat 
valid_iter2= valid_dat




train_dataloader = DataLoader(train_iter2, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(valid_iter2, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_iter2, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

| epoch   1 |   500/ 1338 batches | accuracy    0.449
| epoch   1 |  1000/ 1338 batches | accuracy    0.540
-----------------------------------------------------------
| end of epoch   1 | time:  6.79s | valid accuracy    0.589 
-----------------------------------------------------------
| epoch   2 |   500/ 1338 batches | accuracy    0.617
| epoch   2 |  1000/ 1338 batches | accuracy    0.623
-----------------------------------------------------------
| end of epoch   2 | time:  3.99s | valid accuracy    0.616 
-----------------------------------------------------------
| epoch   3 |   500/ 1338 batches | accuracy    0.667
| epoch   3 |  1000/ 1338 batches | accuracy    0.670
-----------------------------------------------------------
| end of epoch   3 | time:  4.28s | valid accuracy    0.627 
-----------------------------------------------------------
| epoch   4 |   500/ 1338 batches | accuracy    0.698
| epoch   4 |  1000/ 1338 batches | accuracy    0.707
-------------------------

In [18]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.
test accuracy    0.679


In [19]:
sentiment_label = {2:"Positive",
                   1: "Negative",
                   0: "Neutral"
                  }

def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() 
ex_text_str = "Happy Birthday Snickers!!!! ? I hope you have the best day ever! Let`s go shopping!!!"
model = model.to("cpu")

print("This is a %s tweet" %sentiment_label[predict(ex_text_str, text_pipeline)])

This is a Positive tweet


In [22]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[ 0.1307,  0.0202, -0.1635,  ...,  0.4103, -0.1344,  0.3930],
        [-0.1276, -0.0887,  0.2664,  ..., -0.0921,  0.2704,  0.0501],
        [-0.0218, -0.1828, -0.1208,  ...,  0.2758, -0.2773,  0.0977],
        ...,
        [ 0.2968, -0.3142, -0.3402,  ...,  0.4607,  0.0584, -0.5102],
        [-0.1291, -0.0230,  0.3256,  ..., -0.4332,  0.4607, -0.0858],
        [ 0.0022, -0.4016, -0.0677,  ..., -0.3719, -0.1085, -0.4813]],
       requires_grad=True)
Parameter containing:
tensor([[-4.1814e-01,  2.6218e-01, -3.5425e-01,  ..., -9.6711e-02,
          1.2056e-01,  2.3817e-01],
        [-6.3517e-01,  3.2474e-01,  2.6492e-01,  ...,  1.7044e-01,
          7.1621e-04, -8.2756e-02],
        [-3.4442e-01, -8.5487e-01,  1.2719e+00,  ..., -1.3776e-01,
          4.5986e-01, -1.2128e-01],
        ...,
        [-3.2299e-01,  4.4040e-01,  5.7413e-01,  ...,  2.8853e-01,
         -3.6174e-01, -3.8113e-01],
        [-2.2013e-01, -1.1470e+00,  6.8965e-01,  ..., -5.6169e-01,
   

In [20]:
FILE = "model.pth"
torch.save(model, FILE)

In [13]:
FILE = "model.pth"

model = torch.load(FILE)
model.eval()

# for param in model.parameters():
#     print(param)

sentiment_label = {2:"Positive",
                   1: "Negative",
                   0: "Neutral"
                  }

def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() 
ex_text_str = "Happy Birthday Snickers!!!! ? I hope you have the best day ever! Let`s go shopping!!!"
model = model.to("cpu")

print("This is a %s tweet" %sentiment_label[predict(ex_text_str, text_pipeline)])

This is a Positive tweet
